# IBM - Natural Language Understanding

Install the required IBM software to communicate with the API

In [3]:
!pip install --upgrade "watson-developer-cloud>=2.4.1"

  Running setup.py bdist_wheel for watson-developer-cloud: started
  Running setup.py bdist_wheel for watson-developer-cloud: finished with status 'done'
  Stored in directory: C:\Users\divya\AppData\Local\pip\Cache\wheels\36\21\38\6eaa2c266b1ac5ce6acb09c13766385c4169665dd9c8b3bd35
Successfully built watson-developer-cloud


Import related libraries

In [8]:
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, SentimentOptions

In [12]:
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-03-16',
    iam_apikey='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',
    url='https://gateway.watsonplatform.net/natural-language-understanding/api'
)



Extract just the paragraphs from our call earning dataset

In [2]:
import json
import csv
import pandas as pd

In [69]:
df = pd.read_csv("call_transcript.csv")

In [19]:
df.head(5)

,paragraph,sentiment
0,"Good day ladies and gentlemen, and welcome to ...",neutral
1,Thank you. Good afternoon everyone and welcome...,neutral
2,G&A expense trends in the third quarter were a...,neutral
3,"To give you a bit of a breakdown, the largest ...",positive
4,"That being said, we do remain very focused on ...",neutral


In [20]:
df.shape

(622, 2)

In [42]:
IBM_sentiment = []
for x in df['paragraph']:
    response = natural_language_understanding.analyze(
    text= x,
    features=Features(sentiment=SentimentOptions())).get_result()
    IBM_sentiment.append(response)

In [43]:
type(IBM_sentiment)

list

In [44]:
sentiment_dumps = json.dumps(IBM_sentiment, indent =3)

write_json= open("IBM_Api_sentiments.json","w")
write_json.write(sentiment_dumps)
write_json.close()

# Microsoft - Text Analytics

In [1]:
import requests
from pprint import pprint

In [2]:
subscription_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
assert subscription_key

In [3]:
text_analytics_base_url = "https://eastus.api.cognitive.microsoft.com/text/analytics/v2.0/"

In [4]:
sentiment_api_url = text_analytics_base_url + "sentiment"
print(sentiment_api_url)

https://eastus.api.cognitive.microsoft.com/text/analytics/v2.0/sentiment


In [3]:
df = pd.read_csv("call_transcript.csv")

In [4]:
df = df[['paragraph']]

In [9]:
df['language'] = 'en'

In [10]:
df['id'] = df.index

In [14]:
df = df.rename(columns={'paragraph': 'text'})
df = df[['id','language','text']]

In [15]:
para = df.to_dict(orient='records')

In [50]:
documents = {'documents':para}

In [53]:
headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
response  = requests.post(sentiment_api_url, headers=headers, json=documents)
sentiments = response.json()
pprint(sentiments)

{'documents': [{'id': '0', 'score': 0.9581071138381958},
               {'id': '1', 'score': 0.9617360830307007},
               {'id': '2', 'score': 0.5},
               {'id': '3', 'score': 0.5},
               {'id': '4', 'score': 0.5},
               {'id': '5', 'score': 0.5},
               {'id': '6', 'score': 0.9667835235595703},
               {'id': '7', 'score': 0.8553870320320129},
               {'id': '8', 'score': 0.9268252849578857},
               {'id': '9', 'score': 0.5},
               {'id': '10', 'score': 0.8392119407653809},
               {'id': '11', 'score': 0.9328951835632324},
               {'id': '12', 'score': 0.9667835235595703},
               {'id': '13', 'score': 0.7972878813743591},
               {'id': '14', 'score': 0.935219407081604},
               {'id': '15', 'score': 0.9253172278404236},
               {'id': '16', 'score': 0.5},
               {'id': '17', 'score': 0.5},
               {'id': '18', 'score': 0.7079541683197021},
              

In [58]:
import json
with open('Azure_Api_sentiments.json', 'w') as fp:
    json.dump(sentiments, fp)

# AWS - Amazon Comprehend

In [59]:
!pip install awscli

  Using cached https://files.pythonhosted.org/packages/ef/cc/d8f288b5b450e59b92c460727012ee002925324bc655255a9945a45ab9ad/awscli-1.16.59-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e1/ae/baedc9cb175552e95f3395c43055a6a5e125ae4d48a1d7a924baca83e92e/rsa-3.4.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9f/92/ef02dcb2ba6bf95e96b68884a94d3e66e81093aac3b358d95898f07fda20/botocore-1.12.49-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d7/14/2a0004d487464d120c9fb85313a75cd3d71a7506955be458eebfe19a6b1d/s3transfer-0.1.13-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b7/31/05c8d001f7f87f0f07289a5fc0fc3832e9a57f2dbd4d3b0fee70e0d51365/jmespath-0.9.3-py2.py3-none-any.whl


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [61]:
!pip install boto3

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import boto3
import json

In [20]:
df.tail(5)

,paragraph
617,"Hopefully, you've seen already Jack Ryan and T..."
618,This is Dave. Just on the AWS pricing philosop...
619,(30:42) call today and for your questions. A r...
620,"Our guidance also assumes, among other things,..."
621,"With that, we'll move to Q&A. Operator, please..."


In [21]:
comprehend = boto3.client(service_name='comprehend', region_name ='us-east-1',
                          aws_access_key_id="xxxxxxxxxxxxxxxxxxxxx", aws_secret_access_key="xxxxxxxxxxxxxxxxxxxxxxxxx")

In [22]:
AWS_sentiment = []
for x in df['paragraph']:
    response = json.dumps(comprehend.detect_sentiment(Text=x, LanguageCode='en'), sort_keys=True, indent=4)
    print(x)
    print(response)
    AWS_sentiment.append(response)

Good day ladies and gentlemen, and welcome to the Alphabet Third Quarter 2018 Earnings Call. I'd now like to turn the conference over to Ellen West, Head of Investor Relations. Please go ahead.
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "163",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 27 Nov 2018 03:48:32 GMT",
            "x-amzn-requestid": "4f31cd98-f1f7-11e8-8656-5735cedac40a"
        },
        "HTTPStatusCode": 200,
        "RequestId": "4f31cd98-f1f7-11e8-8656-5735cedac40a",
        "RetryAttempts": 0
    },
    "Sentiment": "POSITIVE",
    "SentimentScore": {
        "Mixed": 0.010902220383286476,
        "Negative": 0.004799054004251957,
        "Neutral": 0.4700372815132141,
        "Positive": 0.5142614841461182
    }
}
Thank you. Good afternoon everyone and welcome to Alphabet's third quarter 2018 earnings conference call. With us today are Ruth Porat an

And on your second question, one of the things we clearly see when we make a hardware product like Pixel, in which all the tools and the utilities we build are conveniently there integrated and the experience is great. We definitely see users engaging more. And so, we see an opportunity. And that's one of the bigger reasons why we do hardware as well, to show that into an experience, both for our ecosystem as well as for us. It helps us give users a much deeper engaged experience as well. And when you look at all our products, we see that. And so, we do see that as an opportunity.
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "164",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 27 Nov 2018 03:48:33 GMT",
            "x-amzn-requestid": "4fd9312d-f1f7-11e8-9915-fbeda43b2a4c"
        },
        "HTTPStatusCode": 200,
        "RequestId": "4fd9312d-f1f7-11e8-9915-fbeda43b2a4c

And also if you could highlight, if you've noticed if there's been noticeable changes in win rates over the last year as the product continues to mature. And you, also in the beginning of the year and exiting Q4, you would give us some high-level growth commentary about GCP. and I'm just wondering if you have anything else you could share? Thank you.
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "164",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 27 Nov 2018 03:48:34 GMT",
            "x-amzn-requestid": "507ac87c-f1f7-11e8-b065-45a3bc36c640"
        },
        "HTTPStatusCode": 200,
        "RequestId": "507ac87c-f1f7-11e8-b065-45a3bc36c640",
        "RetryAttempts": 0
    },
    "Sentiment": "POSITIVE",
    "SentimentScore": {
        "Mixed": 0.044872578233480453,
        "Negative": 0.017318986356258392,
        "Neutral": 0.16220778226852417,
        "Positive": 0.77

Thanks. Ruth, I just wanted to see if you could quantify the FX headwind. I think it was a negative 1% for Q3 and in Q4, do you anticipate it to be similar or a little worse?
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "162",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 27 Nov 2018 03:48:35 GMT",
            "x-amzn-requestid": "50efd08d-f1f7-11e8-983f-d98c2036a672"
        },
        "HTTPStatusCode": 200,
        "RequestId": "50efd08d-f1f7-11e8-983f-d98c2036a672",
        "RetryAttempts": 0
    },
    "Sentiment": "NEGATIVE",
    "SentimentScore": {
        "Mixed": 0.2395295947790146,
        "Negative": 0.33862990140914917,
        "Neutral": 0.31471753120422363,
        "Positive": 0.10712291300296783
    }
}
I will let you forecast the dollar. As you noted, it was 1 point here. We went from a tailwind in the second quarter to a 1 point drag here going forward. Bu

Look, I mean, I think on the first thing on e-commerce, maybe really in people, we do see a lot of activity in the vertical on our products, and we see strong growth there as well. We see is an important use case and that's why we are investing a lot. And if you look at our recent work with Shopping Actions, that's an example of the kind of work we are doing there.
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "163",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 27 Nov 2018 03:48:36 GMT",
            "x-amzn-requestid": "5169947e-f1f7-11e8-82a8-359a9a64fb2f"
        },
        "HTTPStatusCode": 200,
        "RequestId": "5169947e-f1f7-11e8-82a8-359a9a64fb2f",
        "RetryAttempts": 0
    },
    "Sentiment": "NEUTRAL",
    "SentimentScore": {
        "Mixed": 0.010838746093213558,
        "Negative": 0.008618759922683239,
        "Neutral": 0.8460405468940735,
        "Po

...

Sure. Yeah, I think we've started to show our strategy on the Whole Foods side. You'll see even this quarter we started to have greater expansion of our grocery delivery out of Whole Foods using Prime Now. We're now in 60 cities in the U.S., giving customers delivery in as fast as an hour on thousands of great organic products from Whole Foods.
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "161",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 27 Nov 2018 03:49:27 GMT",
            "x-amzn-requestid": "6fde3624-f1f7-11e8-8560-532dc7aa62ea"
        },
        "HTTPStatusCode": 200,
        "RequestId": "6fde3624-f1f7-11e8-8560-532dc7aa62ea",
        "RetryAttempts": 0
    },
    "Sentiment": "NEUTRAL",
    "SentimentScore": {
        "Mixed": 0.007290900219231844,
        "Negative": 0.0339224711060524,
        "Neutral": 0.6119139194488525,
        "Positive": 0.346872627735

Thank you. Our next question comes from the line of Youssef Squali with SunTrust Robinson Humphrey. Please proceed.
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "161",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 27 Nov 2018 03:49:27 GMT",
            "x-amzn-requestid": "70569b08-f1f7-11e8-b966-5f330c378b33"
        },
        "HTTPStatusCode": 200,
        "RequestId": "70569b08-f1f7-11e8-b966-5f330c378b33",
        "RetryAttempts": 0
    },
    "Sentiment": "POSITIVE",
    "SentimentScore": {
        "Mixed": 0.03997579589486122,
        "Negative": 0.13135658204555511,
        "Neutral": 0.1143055260181427,
        "Positive": 0.7143620848655701
    }
}
Our comments and responses to your questions reflect management's views as of today, October 25, 2018 only, and will include forward-looking statements. Actual results may differ materially. Additional information abo

Yeah. Thanks, Jason. This is Dave. Just quickly on the ASC impact, Brian mentioned earlier that of course we've been talking about it all year the changing, and the subscription revenue recognition is now straight line. So on a comp basis you'd expect to see subscription revenue be about $300 million lower due to that accounting change.
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "163",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 27 Nov 2018 03:49:28 GMT",
            "x-amzn-requestid": "70d0375d-f1f7-11e8-a4e1-2d227516436e"
        },
        "HTTPStatusCode": 200,
        "RequestId": "70d0375d-f1f7-11e8-a4e1-2d227516436e",
        "RetryAttempts": 0
    },
    "Sentiment": "NEUTRAL",
    "SentimentScore": {
        "Mixed": 0.020587198436260223,
        "Negative": 0.033603258430957794,
        "Neutral": 0.8434197306632996,
        "Positive": 0.10238980501890182


If we do this right, we think we'll both help advertisers and help Amazon consumers at the same time. So I know it doesn't answer the question about specific rate increases or capacity, but that is the general strategy that we're seeing in advertising.
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "160",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 27 Nov 2018 03:49:29 GMT",
            "x-amzn-requestid": "714c1e27-f1f7-11e8-b11b-a5517c2961dd"
        },
        "HTTPStatusCode": 200,
        "RequestId": "714c1e27-f1f7-11e8-b11b-a5517c2961dd",
        "RetryAttempts": 0
    },
    "Sentiment": "NEUTRAL",
    "SentimentScore": {
        "Mixed": 0.03719165548682213,
        "Negative": 0.24843774735927582,
        "Neutral": 0.627427875995636,
        "Positive": 0.08694270998239517
    }
}
Yeah, and this is Dave. Just jumping on the Alexa point, I mean, I think the shor

Great. Thanks. I guess two quick ones. Any change in the trajectory of Prime membership growth since the June price increase? And then, how would you describe the pricing environments for the core AWS services? And is this still a key lever that you use for business development? Thank you.
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "165",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 27 Nov 2018 03:49:30 GMT",
            "x-amzn-requestid": "71b62a3b-f1f7-11e8-b280-a343d0954400"
        },
        "HTTPStatusCode": 200,
        "RequestId": "71b62a3b-f1f7-11e8-b280-a343d0954400",
        "RetryAttempts": 0
    },
    "Sentiment": "POSITIVE",
    "SentimentScore": {
        "Mixed": 0.007728257216513157,
        "Negative": 0.00045028357999399304,
        "Neutral": 0.012403234839439392,
        "Positive": 0.979418158531189
    }
}
Yeah. Sure. Let me start with the Ama

In [27]:
len(AWS_sentiment)

622

In [28]:
sentiment_dumps = json.dumps(AWS_sentiment, indent =4)

write_json= open("AWS_Api_sentiments.json","w")
write_json.write(sentiment_dumps)
write_json.close()